A sample script:

```
../../bronco/bin/dev train --showLogs -p batches_per_epoch=20 -p epochs=500 -p model=rdn -p C=4 -p D=3 -p G=4 -p G0=64 -p lr=0.0004 --verbose -p hr_patch_size=128 -p scale=2 -e local -p dataset=div2kpreprocessed -p with_pruning=True -i /mnt/castellan/data/div2k-processed/preprocessed/2x
```

In [1]:
import pathlib
import os
from IPython.display import display

batches_per_epoch = 20
epochs = 4000
lr = 0.0004

USE_GPU = True

batch_size = 8

scripts = []

def make_script(script_parts):
    return ' '.join(script_parts)
    script_parts = [script_parts[0]] + [f'    {p}' for p in script_parts[1:]]
    return ' \ \n'.join(script_parts)

for dataset_name, dataset_path in [
    ('div2k', pathlib.Path('/mnt/castellan/data/div2k-processed/preprocessed')),
    # ('ffhq', pathlib.Path('/mnt/castellan/data/ffhq/preprocessed')),
    # ('div2k_flickr2k', pathlib.Path('/mnt/castellan/data/div2k-flickr2k/preprocessed')),
    # ('div2k_ffhq_flickr2k', pathlib.Path('/mnt/castellan/data/div2k-ffhq-flickr2k/preprocessed')),
]:
    for compression, sharpen in [
            (None, None),
            # (True, None),
            # (None, True),
            # (True, True),
        ]:

        for arch, C, D, G, G0 in [
            #small
            ('rdn', 1, 2, 4, 64),
            # #medium
            # ('rdn', 1, 10, 64, 64),
            # # large
            # ('rrdn', 4, 3, 32, 64),
        ]:
            for scale, hr_patch_size in [
                (2, 128), 
                # (3, 129), 
                # (4, 128),
                # (8, 128),
            ]:
                hr_patch_size *= 1
                scale_dataset_path = dataset_path / f'{scale}x'
                gpu = '--gpu' if USE_GPU else ''
                script_parts = [
                    '../bronco/bin/dev',
                    'train',
                    '--showLogs',
                    '-a',
                    '-e local',
                    '--verbose',
                    f'-i {scale_dataset_path}',
                    f'-p batches_per_epoch={batches_per_epoch}',
                    f'-p epochs={epochs}',
                    f'-p model={arch}',
                    f'-p C={C}',
                    f'-p D={D}',
                    f'-p G={G}',
                    f'-p G0={G0}',
                    f'-p lr={lr}',
                    f'-p hr_patch_size={hr_patch_size}',
                    f'-p scale={scale}',
                    f'-p dataset={dataset_name}',
                    f'-p batch_size={batch_size}',
                    gpu,
                ]
                if compression is not None:
                    script_parts.append(f'-p compression_quality=50')
                    script_parts.append(f'-p vary_compression_quality=1')
                if sharpen is not None:
                    script_parts.append(f'-p sharpen_amount=1')
                
                scripts.append(make_script(script_parts))

all_scripts = []
for i, script in enumerate(scripts):
    all_scripts.append(make_script([f'echo "Running script {i + 1} of {len(scripts)}"']))
    all_scripts.append(script)
script_command = "\n".join(all_scripts)

# with open('../run_training_commands.sh', 'w') as f:
#     f.write(f'#!/bin/bash\n\n{script_command}')

# !chmod +x ../run_training_commands.sh

print(script_command)


echo "Running script 1 of 1"
../bronco/bin/dev train --showLogs -a -e local --verbose -i /mnt/castellan/data/div2k-processed/preprocessed/2x -p batches_per_epoch=20 -p epochs=4000 -p model=rdn -p C=1 -p D=2 -p G=4 -p G0=64 -p lr=0.0004 -p hr_patch_size=128 -p scale=2 -p dataset=div2k -p batch_size=8 --gpu


# Generating Experiments of different hyperparameters

In [7]:
import pathlib
import math
import os
from IPython.display import display

TOTAL_N = 500 * 20 * 12
batches_per_epoch = 20
batch_size = 8
epochs = math.ceil(TOTAL_N / batch_size / batches_per_epoch)
lr = 0.0004

USE_GPU = True



scripts = []

def make_script(script_parts):
    return ' '.join(script_parts)
    script_parts = [script_parts[0]] + [f'    {p}' for p in script_parts[1:]]
    return ' \ \n'.join(script_parts)

dataset_name = 'div2k'
dataset_path = pathlib.Path('/mnt/castellan/data/div2k-processed/preprocessed')
for arch, T in [
    ('rdn', 0), 
    # ('rrdn', 1),
    # ('rrdn', 2),
    # ('rrdn', 4),
    # ('rrdn', 8),
    # ('rrdn', 16),
]:
    for C in [1, 2, 4, 8]:
        for D in [1, 2, 4, 8, 16]:
            for G in [2, 4, 8, 16, 32, 64]:
                for G0 in [64]:
                    scale = 2
                    hr_patch_size = 128
                    scale_dataset_path = dataset_path / f'{scale}x'
                    gpu = '--gpu' if USE_GPU else ''
                    script_parts = [
                        '../bronco/bin/dev',
                        'train',
                        '--showLogs',
                        '-a',
                        '-e local',
                        '--verbose',
                        f'-i {scale_dataset_path}',
                        f'-p batches_per_epoch={batches_per_epoch}',
                        f'-p epochs={epochs}',
                        f'-p model={arch}',
                        f'-p C={C}',
                        f'-p D={D}',
                        f'-p G={G}',
                        f'-p G0={G0}',
                        f'-p T={T}',
                        f'-p lr={lr}',
                        f'-p hr_patch_size={hr_patch_size}',
                        f'-p scale={scale}',
                        f'-p dataset={dataset_name}',
                        f'-p batch_size={batch_size}',
                        gpu,
                    ]

                    scripts.append(make_script(script_parts))

all_scripts = []
for i, script in enumerate(scripts):
    all_scripts.append(make_script([f'echo "Running script {i + 1} of {len(scripts)}"']))
    all_scripts.append(script)
script_command = "\n".join(all_scripts)

print(script_command)


echo "Running script 1 of 120"
../bronco/bin/dev train --showLogs -a -e local --verbose -i /mnt/castellan/data/div2k-processed/preprocessed/2x -p batches_per_epoch=20 -p epochs=750 -p model=rdn -p C=1 -p D=1 -p G=2 -p G0=64 -p T=0 -p lr=0.0004 -p hr_patch_size=128 -p scale=2 -p dataset=div2k -p batch_size=8 --gpu
echo "Running script 2 of 120"
../bronco/bin/dev train --showLogs -a -e local --verbose -i /mnt/castellan/data/div2k-processed/preprocessed/2x -p batches_per_epoch=20 -p epochs=750 -p model=rdn -p C=1 -p D=1 -p G=4 -p G0=64 -p T=0 -p lr=0.0004 -p hr_patch_size=128 -p scale=2 -p dataset=div2k -p batch_size=8 --gpu
echo "Running script 3 of 120"
../bronco/bin/dev train --showLogs -a -e local --verbose -i /mnt/castellan/data/div2k-processed/preprocessed/2x -p batches_per_epoch=20 -p epochs=750 -p model=rdn -p C=1 -p D=1 -p G=8 -p G0=64 -p T=0 -p lr=0.0004 -p hr_patch_size=128 -p scale=2 -p dataset=div2k -p batch_size=8 --gpu
echo "Running script 4 of 120"
../bronco/bin/dev train 